# Movie_Database1

INTRODUCTION OF DATABASE.

In this project, i'm going to working on
an online, free movie and TV show database that houses some of the most popular movies and TV shows worldwide. 
The TMDb database supports 39 official languages used in over 180 countries daily and dates back all the way to 2008. 

The database can be accessed freeily for learning puporse here https://www.themoviedb.org/ . 

1.EXPLORE DATABASE


In [12]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [11]:
#starting a connection to a local database using sql magic command
%sql sqlite:///TMDB-a-40061.db

'Connected: @TMDB-a-40061.db'

what Is is inside our database?

As can be seen from the ERD (https://www.themoviedb.org/), the TMDb database consists of 12 tables containing information about 
movies, cast, genre, and so much more.


Let's start exploring the Database deeply

I am firstly interested to know who won oscar prize "Actor in a Leading Role" recently?


To know that, let me check what are the recent years firstly.

In [16]:
%%sql
-- 1.0 mention winner and mention the award name
select * 
from oscars
where winner = '1.0' AND award = 'Actor in a Leading Role' 
order by year DESC -- to look for earliest
limit 1;

   sqlite:///TMDB-a-40061
 * sqlite:///TMDB-a-40061.db
Done.


year,award,winner,name,film
2015,Actor in a Leading Role,1.0,Leonardo DiCaprio,The Revenant


also we want to know the oldest movie in our database

In [23]:
%%sql 
select movie_id, title, release_date, budget 
from movies
where release_date IS NOT NULL
order by release_date 
limit 2;

   sqlite:///TMDB-a-40061
 * sqlite:///TMDB-a-40061.db
Done.


movie_id,title,release_date,budget
3059,Intolerance,1916-09-04 00:00:00.000000,385907
3060,The Big Parade,1925-11-05 00:00:00.000000,245000


One of our sponsor is curious to know the total number unique awards are there in the Oscars table

In [27]:
%%sql 
select COUNT (distinct award) as Number_of_awards
from oscars
limit 5;

   sqlite:///TMDB-a-40061
 * sqlite:///TMDB-a-40061.db
Done.


Number_of_awards
114


In [33]:
%%sql
select movie_id, title, release_date
from movies
where title LIKE '%spider%' ;

   sqlite:///TMDB-a-40061
 * sqlite:///TMDB-a-40061.db
Done.


movie_id,title,release_date
557,Spider-Man,2002-05-01 00:00:00.000000
558,Spider-Man 2,2004-06-25 00:00:00.000000
559,Spider-Man 3,2007-05-01 00:00:00.000000
1930,The Amazing Spider-Man,2012-06-27 00:00:00.000000
2043,Along Came a Spider,2001-04-06 00:00:00.000000
8204,The Spiderwick Chronicles,2008-02-14 00:00:00.000000
9613,Spider,2002-12-13 00:00:00.000000
30315,Kingdom of the Spiders,1977-08-24 00:00:00.000000
102382,The Amazing Spider-Man 2,2014-04-16 00:00:00.000000


Lets count those movie

In [35]:
%%sql
select count (distinct title) AS nbr_of_spiders
from (select movie_id, title, release_date
from movies
where title LIKE '%spider%');

   sqlite:///TMDB-a-40061
 * sqlite:///TMDB-a-40061.db
Done.


nbr_of_spiders
9


In [36]:
How many movies are there that are both in the "Thriller" genre and contain the word “love” anywhere in the keywords?

Object `keywords` not found.


In [38]:
%%sql 
select * from genres
limit 2;

   sqlite:///TMDB-a-40061
 * sqlite:///TMDB-a-40061.db
Done.


genre_id,genre_name
12,Adventure
14,Fantasy


In [ ]:
How many movies are there that are both in the "Thriller" genre and contain the word “love” anywhere in the keywords?

In [15]:
%%sql 
SELECT COUNT(distinct movie_id) AS nbrThrillerLove
FROM (
    SELECT m.title, g.genre_id, ge.genre_name, m.movie_id 
    FROM Movies m
    JOIN genremap g ON g.movie_id = m.movie_id
    JOIN genres ge ON ge.genre_id = g.genre_id
    JOIN keywordmap km ON km.movie_id = m.movie_id
    JOIN keywords ke ON ke.keyword_id = km.keyword_id
    WHERE ge.genre_name = 'Thriller' AND ke.keyword_name LIKE '%love%'
) AS subquery;


 * sqlite:///TMDB-a-40061.db
Done.


nbrThrillerLove
48


let's look at gender difference in actors. 

How many female actors (i.e. gender = 1)

In [17]:
%%sql
select * 
from actors
where gender LIKE 1
limit 5;

 * sqlite:///TMDB-a-40061.db
Done.


actor_id,actor_name,gender
4,Carrie Fisher,1
14,Ellen DeGeneres,1
19,Allison Janney,1
20,Elizabeth Perkins,1
32,Robin Wright,1


In [23]:
%%sql
SELECT
    (SELECT COUNT(distinct actor_name) FROM actors WHERE gender = 1) AS number_of_females,
    (SELECT COUNT(distinct actor_name) FROM actors WHERE gender = 2) AS number_of_males;


 * sqlite:///TMDB-a-40061.db
Done.


number_of_females,number_of_males
10441,16317



There is a company named “The Royal Tenenbaums”, we need to know its movies genres

In [25]:
%%sql 
SELECT m.movie_id, m.title
FROM movies m
JOIN genremap gm ON m.movie_id = gm.movie_id
JOIN genres g ON gm.genre_id = g.genre_id
WHERE g.genre_id IN (18, 35)
  OR m.title = 'The Royal Tenenbaums'
    limit 2;


 * sqlite:///TMDB-a-40061.db
Done.


movie_id,title
13,Forrest Gump
14,American Beauty


What are the first three company with highest popularity on average?

In [27]:
%%sql
SELECT
  distinct pc.production_company_name, pcm.production_company_id, AVG(m.popularity) OVER (PARTITION BY pcm.production_company_id) AS avgPop
FROM
  movies m
JOIN
  productioncompanymap pcm ON pcm.movie_id = m.movie_id
JOIN
  productioncompanies pc ON pc.production_company_id = pcm.production_company_id
order by avgPop DESC
LIMIT 3;

 * sqlite:///TMDB-a-40061.db
Done.


production_company_name,production_company_id,avgPop
The Donners' Company,11307,514.569956
Bulletproof Cupid,54850,481.098624
Kinberg Genre,78091,326.92099900000005


Which genre has, on average, the lowest movie popularity score?

In [28]:
%%sql
select distinct g.genre_id, g.genre_name, Avg(m.popularity) over (partition by genre_name) AS avg_Popularity  from genres g
join genremap gm ON g.genre_id=gm.genre_id
join movies m ON gm.movie_id=m.movie_id
order by avg_Popularity
limit 3;


 * sqlite:///TMDB-a-40061.db
Done.


genre_id,genre_name,avg_Popularity
10769,Foreign,0.686786794117647
99,Documentary,3.945723809090908
10770,TV Movie,6.389414749999999


 Which award category has the highest number of actor nominations (actors can be male or female)? (Hint: Oscars.name contains both actors' names and film names.



In [29]:
%%sql
SELECT distinct award, COUNT(name) over (partition by award) AS actor_nominations
FROM oscars
ORDER BY actor_nominations DESC
LIMIT 3;

 * sqlite:///TMDB-a-40061.db
Done.


award,actor_nominations
Directing,429
Film Editing,410
Actor in a Supporting Role,400
